### This train type uses collate function, batch is padded with padding.

In [5]:
# # !pip3 install -q -U huggingface-hub==0.13.0
# !pip3 install -q -U graphviz==0.20.1
# !pip3 install -q torchview==0.2.6

In [6]:
# import sys
# sys.path = ['../input/google-ai-utils'] + sys.path

In [7]:
# import graphviz
# from torchview import draw_graph
# graphviz.set_jupyter_format('png')

In [8]:
import utils

In [9]:
import pandas as pd
import torch
import random
import os
# from utils import create_label
import numpy as np

# coding=utf-8
import numpy as np
import pandas as pd
import os, random, time, gc, argparse, shutil
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
from torch.autograd import Variable
from torch.optim.lr_scheduler import _LRScheduler, ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data.sampler import Sampler
from transformers import get_constant_schedule_with_warmup, AdamW, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer
from transformers import AutoConfig, AutoModel, AutoModelForMaskedLM
import warnings

## Config

In [10]:
parser = argparse.ArgumentParser(description='PyTorch')
parser.add_argument('--out_dir', default='', type=str, help='destination where trained network should be saved')
parser.add_argument('--gpu_id', default='0', type=str, help='gpu id used for training')
parser.add_argument('--model_name', default='deberta-v3-base', type=str)
parser.add_argument('--base_epoch', default=3, type=int, help='number of total epochs to run')#30
parser.add_argument('--batch_size', default=2, type=int, help='train mini-batch size')# 8
parser.add_argument('--workers', default=2, type=int, help='number of data loading workers (default: 4)')

# start lr=1849e-7 and peak lr=1849e-10
parser.add_argument('--lr', default=1849e-10, type=float, help='learning rate')
parser.add_argument('--eta_min', default=1800e-10, type=float, help='learning rate eta_min') 

# without n_accumulate=5 out of memory error even at self.seq_length=128.
parser.add_argument('--n_accumulate', default=5, type=int) 

parser.add_argument('--max_grad_norm', default=-1, type=float) # 1.0
parser.add_argument('--weight_decay', default=0.0, type=float)
parser.add_argument('--folds', default='', type=str)
parser.add_argument('--pre_epoch', default=15, type=int)

# args = parser.parse_args()
args, unknown = parser.parse_known_args()

In [11]:
class Parameter(object):
    def __init__(self):
        # data
        self.result_dir = './models/' # ./user_data/
        self.result_dir_1 = './kaggle/'        
        self.data_dir = './'
        self.model_dir = './'
        self.k_folds = 5
        self.random_seed = 27
        self.seq_length = 2048 # 512; sum of "cell length" for a particular id.
        self.cell_max_length = 128 # max words in a cell.
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        # model
        self.use_cuda = torch.cuda.is_available()
        self.gpu = 0
        self.print_freq = 500
        self.weight_decay = 0
        self.optim = 'Adam'

    def get(self, name):
        return getattr(self, name)

    def set(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

    def __str__(self):
        return '\n'.join(['%s:%s' % item for item in self.__dict__.items()])
    
parameter = Parameter()
parameter.set(**args.__dict__)

In [12]:
def seed_everything(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)
#     if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    #         torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False        
    torch.backends.cudnn.deterministic = True

## Classes

In [13]:
class MarkdownDataset(Dataset):

    def __init__(self, meta_data: pd.DataFrame, tokenizer, fold: int = -1, mode='train', parameter=None):
        self.meta_data = meta_data.copy()
        if mode == 'train':
            pass
            self.meta_data = self.meta_data[self.meta_data['fold_flag'] != fold].copy()
#             self.meta_data = self.meta_data.iloc[:60000]
            self.meta_data.reset_index(drop=True, inplace=True)
        elif mode == 'valid':
            self.meta_data = self.meta_data[self.meta_data['fold_flag'] == fold].copy()
#             self.meta_data = self.meta_data[self.meta_data['id'].isin(self.meta_data['id'].values[:1000])]
            self.meta_data.reset_index(drop=True, inplace=True)
        elif mode == 'test':
            pass
        else:
            raise ValueError(mode)

        if tokenizer.sep_token != '[SEP]':
            self.meta_data['source'] = self.meta_data['source'].apply(
                lambda x: [
                    y.replace(tokenizer.sep_token, '').replace(tokenizer.cls_token, '').replace(tokenizer.pad_token, '')
                    for y in x])
        self.parameter = parameter
        self.seq_length = self.parameter.seq_length
        self.source = self.meta_data['source'].values
        self.cell_type = self.meta_data['cell_type'].values
        # self.cell_id = self.meta_data['cell_id'].values
        self.rank = self.meta_data['rank'].values
        # self.dense_features = self.meta_data[['cell_count','markdown_count', 'code_count']].values
        self.mode = mode
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        source = self.source[index]
            # (no data cleaning) source =>
            # array([' this python 3 environment .... the current session', ..., ' на обучении на один признак больше чем на тесте '], dtype=object)
            
            # [len(x) for x in source] => 
            # [845, 491, 47, 40, 555, 154, 54, 15, 141, 51, 74, 35, ..., 62, 34, 39, 49]            
        
            # max([len(x) for x in source]), min([len(x) for x in source]) => 3722, 10
        cell_type = self.cell_type[index]
            # cell_type =>
            # array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ....,  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])        
        
        rank = self.rank[index]
            # rank => array([0.01724138, 0.05172414, 0.0862069 , 0.12068966, 0.15517241, ..., 0.93103448, 0.62068966, 0.4137931 ]) 
    
        # dense_features = 1#self.dense_features[index]
        # if self.mode == 'train':
        #     range_tmp1 = [ i for i in range(len(cell_type)) if cell_type[i]==0]
        #     range_tmp2 = [ i for i in range(len(cell_type)) if cell_type[i]==1]
        #     np.random.shuffle(range_tmp2)
        #     source = [source[i] for i in range_tmp1 + range_tmp2]
        #     rank = [rank[i] for i in range_tmp1 + range_tmp2]
        #     rank2 = [rank2[i] for i in range_tmp1 + range_tmp2]

        cell_inputs = self.tokenizer(#.batch_encode_plus
            source.tolist(),
            add_special_tokens=False,
            max_length=self.parameter.cell_max_length,
            # padding="max_length",
            return_attention_mask=False,
            truncation=True,
        )
            # cell_inputs =>
            # {'input_ids': [[291, 24233, 404, ..., 2, 763, 5514, 4219], ..., [15003, 42120, 69819, ..., 15003, 65639, 109249]],
            # 'token_type_ids': [[0, 0, 0, ..., 0, 0, 0, 0], ..., [0, 0, 0, ..., 0, 0, 0]]}

            # [len(x) for x in cell_inputs['input_ids']] => [128, 105, 7, 10, 127, 28, 12, 4, 26, 12, 23, 9, ... , 18]        
            # max([len(x) for x in cell_inputs['input_ids']]), min([len(x) for x in cell_inputs['input_ids']]) => 128, 4        
        
        seq, seq_mask, target_mask, target = self.max_length_rule_base(cell_inputs['input_ids'], cell_type, rank)
        

        # if self.mode == 'train':
        #     attention_mask, target = self.random_mask(attention_mask, target)

        return seq, seq_mask, target_mask, target
        # return encoded['input_ids'][0], encoded['attention_mask'][0], np.array(target, dtype=np.float32)

    def __len__(self):
        return len(self.meta_data)

    def max_length_rule_base(self, cell_inputs, cell_type, rank):
        init_length = [len(x) for x in cell_inputs]
            # init_length => [128, 105, 7, ..., 13, 16, 18]
            # self.seq_length => 512
            # len(init_length) => 58
        total_max_length = self.seq_length - len(init_length)
            # total_max_length => 454
        min_length = total_max_length // len(init_length)
            # min_length => 7
        cell_length = self.search_length(init_length, min_length, total_max_length, len(init_length))
            # cell_length => [11, 11, 7, ..., 7, 7, 7]             
            # len(cell_length) => 58
        
        seq = []
        for i in range(len(cell_length)):
            if cell_type[i] == 0:
                seq.append(self.tokenizer.cls_token_id)
                # self.tokenizer.cls_token_id => 1 
            else:
                seq.append(self.tokenizer.sep_token_id)
                # self.tokenizer.sep_token_id => 2
            if cell_length[i] > 0:
                seq.extend(cell_inputs[i][:cell_length[i]])

            # cell_inputs[i] => [291, 24233, 404, ..., 2, 763, 5514, 4219]
            # seq => [1, 291, 24233, 404, 1192, 699, 275, 386, 2136, 7027, 7296, 2627] 
            
        # seq => [1, 291, 24233, 404, 1192, 699, 275, 386, 2136, 7027, 7296, 2627, 1, 6306, ..., 2, 15003, ..., 88580]
        
        # seq_mask or attention_mask tells model which position should it attend in seq. 1 -> attend and 0 -> not attend.    
        if len(seq) > self.seq_length:
            seq_mask = [1] * self.seq_length
            seq = seq[:self.seq_length]
        else: seq_mask = [1] * len(seq) 
            
        # seq_mask is attention_mask.
        seq, seq_mask = np.array(seq, dtype=np.int32), np.array(seq_mask, dtype=np.int32)
        target_mask = np.where((seq == self.tokenizer.cls_token_id) | (seq == self.tokenizer.sep_token_id), 1, 0) 
            # target_mask => [1 0 0 0 0 0 0 0 0 0 0 0 1 ..... 0 0 0 0]
            
        target = np.zeros(len(seq), dtype=np.float32)
        
        # return indices where self.tokenizer.cls_token_id or self.tokenizer.sep_token_id is present
        tmp = np.where((seq == self.tokenizer.cls_token_id) | (seq == self.tokenizer.sep_token_id))
            # tmp => (array([  0,  12,  24,  32,  43,  55,  67,  79,  ..., 479, 487, 495, 503]),)        
        target[tmp] = rank
            # target => [0.01724138 0.         0.         0.    ...      0.        ]
            
#         sample_weight = np.zeros(len(seq), dtype=np.float32)
#         sample_weight = np.where(seq == self.tokenizer.cls_token_id, 0.33, sample_weight)
#         sample_weight = np.where(seq == self.tokenizer.sep_token_id, 1.0, sample_weight)
#         dense_features = np.zeros(self.seq_length, dtype=np.float32)
#         dense_features[tmp] = rank2

        return seq, seq_mask, target_mask, target

    # Static method objects provide a way of defeating the transformation of function objects to method objects.
    # A static method does not receive an implicit first argument. we can even call it from an instance. 
    # A class method receives the class as implicit first argument, just like an instance method receives the instance.
    # The insertion of @staticmethod before a method definition, then, stops an instance from sending itself as an argument.
    @staticmethod
    def search_length(init_length, min_length, total_max_length, cell_count, step=5, max_search_count=50):# step=4
            # init_length => [128, 105, 7, ..., 13, 16, 18] 
            # np.sum(init_length) => 1726
            # min_length, total_max_length, cell_count => 7, 454, 58
            # len(init_length) is cell_count.
        if np.sum(init_length) <= total_max_length:
            return init_length

        res = [min(init_length[i], min_length) for i in range(cell_count)]
            # res => [7, 7, 7, 7, 7, 7, 7, 4, ...., 7, 7, 7]
        for s_i in range(max_search_count):
            tmp = [min(init_length[i], res[i] + step) for i in range(cell_count)]# step=4
                # tmp => [11, 11, 7, 10, 11, 11, 11, 4, ...., 11, 11, 11]
                # np.sum(tmp) => 589
            if np.sum(tmp) < total_max_length:
                res = tmp
            else:
                break
                
        for s_i in range(cell_count):
            tmp = [i for i in res]
                # tmp => [7, 7, 7, 7, 7, 7, 7, 4, ..., 7, 7, 7]
            tmp[s_i] = min(init_length[s_i], res[s_i] + step)
                # tmp[s_i], np.sum(tmp) => 11, 398
            if np.sum(tmp) < total_max_length:
                res = tmp
            else:
                break
                
        # res (after all above execution) => [11, 11, 7, 10, 11, 11, 11, 4, ..., 7, 7, 7]
        return res


In [14]:
# dd = MarkdownDataset(train_df, tokenizer, 0, mode='train', parameter=parameter)
# collate_fn = Collate(tokenizer)
# dl = DataLoader(dd, shuffle=True, batch_size=1,
#                                       num_workers=1, pin_memory=True, worker_init_fn=utils.worker_init_fn,
#                                       collate_fn=collate_fn)

In [15]:
# model = MarkdownModel(utils.get_model_path(args.model_name, res = parameter.model_dir), seq_length=parameter.seq_length, pretrained=True)
# for d in dl:
#     draw_graph(model, input_data = [d[0],d[1]], expand_nested=True, save_graph=True, device='cpu').visual_graph
#     break

In [16]:
class Collate:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
    def __call__(self, batch):
        input_ids = [x[0] for x in batch]
            # x[0] => [     1    291  24233    404   1192    699    275    386   2136   7027.....
        attention_mask = [x[1] for x in batch]
            # x[1] => [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 .....
        target_mask = [x[2] for x in batch]
            # x[2] => [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 .....
        target = [x[3] for x in batch]
            # x[3] => [0.0259366  0.         0.         0.         0.         0. ....
        # sample_weight = [x[4] for x in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in input_ids])
        
            # self.tokenizer.pad_token_id => 0
        # add padding
        input_ids = [list(s) + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in input_ids]
        attention_mask = [list(s) + (batch_max - len(s)) * [0] for s in attention_mask]
        target_mask = [list(s) + (batch_max - len(s)) * [0] for s in target_mask]
        target = [list(s) + (batch_max - len(s)) * [0.] for s in target]
        
        # sample_weight = [list(s) + (batch_max - len(s)) * [0] for s in target]
        
        # convert to tensors
        input_ids = torch.tensor(input_ids, dtype=torch.int32) # torch.long
        attention_mask = torch.tensor(attention_mask, dtype=torch.int32) # torch.long
        target_mask = torch.tensor(target_mask, dtype=torch.int32) # torch.float32
        target = torch.tensor(target, dtype=torch.float32)
        # sample_weight = torch.tensor(sample_weight, dtype=torch.float32)

        return input_ids, attention_mask, target_mask, target

In [17]:
class MarkdownModel(nn.Module):
    def __init__(self, name, num_classes=1, seq_length=96, pretrained=True):
        super(MarkdownModel, self).__init__()
        # self.encoder = AutoModel.from_pretrained(name, attention_probs_dropout_prob=0.1, hidden_dropout_prob=0.1)
        self.config = AutoConfig.from_pretrained(name)
        self.config.attention_probs_dropout_prob = 0.
        self.config.hidden_dropout_prob = 0.
        self.config.max_position_embeddings = 4096 * 2 
        # self.config.output_hidden_states = True
        if pretrained:
            self.encoder = AutoModel.from_pretrained(name, config=self.config, ignore_mismatched_sizes=True)
            # self.encoder = AutoModelForMaskedLM.from_pretrained(name, config=self.config)
        else:
            # self.encoder = AutoModelForMaskedLM.from_config(self.config)
            self.encoder = AutoModel.from_config(self.config)

        # self.encoder = AutoModel.from_pretrained(name)
        # print(self.encoder.__dict__)
        # transformer_layers = 2
        # self.seq_length = seq_length
        # self.transformer_layers = transformer_layers
        self.in_dim = self.encoder.config.hidden_size

#         self.pe = PositionalEncoding(self.in_dim)
#         self.trans = nn.Sequential(
#             *[TransformerBlock(emb_s=64, head_cnt=self.in_dim // 64, dp1=0., dp2=0.) for _ in
#               range(transformer_layers)])
        self.bilstm = nn.LSTM(self.in_dim, self.in_dim, num_layers=1, 
                              dropout=self.config.hidden_dropout_prob, batch_first=True,
                              bidirectional=True)
    
#         self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])
#         hidden = 64
#         dropout = 0.
#         self.sequence = nn.Sequential(
#             # nn.BatchNorm1d(1),
#             nn.Linear(1, hidden),  # todo
#             nn.Dropout(dropout),
#             nn.ReLU(),
#             # nn.BatchNorm1d(hidden),
#             nn.Linear(hidden, hidden),
#             nn.Dropout(dropout),
#             nn.ReLU()
#         )

        self.last_fc = nn.Linear(self.in_dim*2, num_classes)
        # self.fc = nn.LazyLinear(num_classes)
        
        # function is intended to be used to initialize neural network parameters, -
        # - so they all run in torch.no_grad() mode and will not be taken into account by autograd.
        # Fills the input Tensor with values drawn from the normal distribution N(mean,std_square).
        torch.nn.init.normal_(self.last_fc.weight, std=0.02)
        
        self.sig = nn.Sigmoid()

    def forward(self, x, mask):
        x = self.encoder(x, attention_mask=mask)["last_hidden_state"]
        # x = x.reshape(-1, code_count, self.seq_length, self.in_dim).mean(2)
        #         x = torch.sum(x * mask.unsqueeze(-1), dim=1) / torch.sum(mask, dim=1).unsqueeze(-1)
        #         x = x.reshape(-1, code_count, self.in_dim)
        # x = x + self.sequence(dense_features.unsqueeze(-1))

#         prev = None
#         x = self.pe(x)
#         for i in range(self.transformer_layers):
#             # x = x * mask.unsqueeze(-1)
#             x, prev = self.trans[i](x, prev)
        # x = torch.sum(x * mask.unsqueeze(-1), dim=1) / torch.sum(mask, dim=1).unsqueeze(-1)
        # x = torch.cat([x, self.sequence(dense_features.unsqueeze(1)).repeat(1,2048,1)], dim=2)
        # x = x.mean(1)
        x, _ = self.bilstm(x)
        out = self.last_fc(x)
#         for i, dropout in enumerate(self.dropouts):
#             if i == 0:
#                 out = self.last_fc(dropout(x))
#             else:
#                 out += self.last_fc(dropout(x))
#         out /= len(self.dropouts)
        # out = self.sig(out)
        out = out.squeeze(-1)
        return out

## Train

In [18]:
def train(model, train_loader, criterion, scheduler, optimizer, epoch, n_accumulate=1, is_adversial=False, scaler=None):
    batch_time = utils.AverageMeter()
    losses = utils.AverageMeter()
    # switch to train mode
    model.train()
    fgm = FGM(model) if is_adversial else None
    start = time.time()
    k=0    
    for i, batch_data in enumerate(train_loader):
        if parameter.use_cuda:
            batch_data = (t.cuda() for t in batch_data)
        seq, seq_mask, target_mask, target = batch_data
        
        with amp.autocast():
            output = model(seq, seq_mask)
                # output.shape => torch.Size([1, 511])
            loss = criterion(output, target, target_mask)
            
        losses.update(loss.item())
        scaler.scale(loss/n_accumulate).backward()
        if is_adversial:
            # confrontation training
            fgm.attack()  # Add anti-perturbation to embedding
            output = model(seq, seq_mask)
            loss_adv = criterion(output, target, target_mask)
            loss_adv.backward()  # Backpropagation, and on the basis of normal grad, accumulate the gradient of confrontation training.
            fgm.restore()  # Restore embedding parameters
        if args.max_grad_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
        if (k + 1) % n_accumulate == 0:
            scaler.step(optimizer)     
            scaler.update()
            optimizer.zero_grad()
            k=-1
            # scheduler.step()
        k+=1
        batch_time.update(time.time() - start)
        start = time.time()
        
#         if i % parameter.print_freq == 0:
#             log.write('Epoch: [{0}][{1}/{2}], Loss {loss:.4f}\n'.format(epoch, i, len(train_loader), loss=loss.item()))

    return losses.avg, batch_time.sum

In [20]:
def validate(model, criterion, valid_loader):#, code_df_valid):
    batch_time = utils.AverageMeter()
    losses = utils.AverageMeter()
    
    # switch to evaluate mode
    model.eval()
    rank_pred = []
    masks = []
    start = time.time()    
    for i, batch_data in enumerate(valid_loader):
        if parameter.use_cuda:
            batch_data = (t.cuda() for t in batch_data)
        seq, seq_mask, target_mask, target = batch_data
        outputs = model(seq, seq_mask)
        loss = criterion(outputs, target, target_mask)
        losses.update(loss.item())
        
        batch_time.update(time.time() - start)
        start = time.time()        
        
    return losses.avg, batch_time.sum        
#         outputs= outputs.detach().cpu().numpy()
#             # outputs.shape => (1, 213) 
#         target_mask = target_mask.detach().cpu().numpy()        
#             # target_mask.shape => (1, 213)
#         tmp1 = np.zeros((outputs.shape[0], 4096))
#             # tmp1.shape => (1, 4096)
#         tmp1[:, :outputs.shape[1]] = outputs
#         tmp2 = np.zeros((outputs.shape[0], 4096))
#             # tmp2.shape => (1, 4096)
#         tmp2[:, :outputs.shape[1]] = target_mask
#         rank_pred.append(tmp1)
#         masks.append(tmp2)
#         # print(y_pred)
#     rank_pred = np.concatenate(rank_pred)
#     masks = np.concatenate(masks)
#     # print(y_pred.shape)
#     # y_pred = y_pred.reshape((-1, parameter.seq_length))
#     score = utils.get_score(valid_loader.dataset.meta_data.copy(), masks, rank_pred, code_df_valid, parameter.data_dir)
#         # valid_loader.dataset.meta_data => 
#         #                   id  id2  fold_flag  cell_count  markdown_count  code_count                                             cell_id     ...
#         # 0     0001bdd4021779    1          0          13               2          11   
#         # 1     0002115f48f982    1          0           9               1           8    [3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310...    ...
        
#     return rank_pred, score

In [21]:
log_out_dir = os.path.join(parameter.result_dir, 'logs')
os.makedirs(log_out_dir, exist_ok=True)
log_dir = os.path.join(log_out_dir, '{}.txt'.format(args.model_name))
if os.path.exists(log_dir):
    os.remove(log_dir)
log = utils.Logger()
log.open(log_dir, mode='a')

model_dir = os.path.join(parameter.result_dir, '')
model_dir_seg = os.path.join(parameter.result_dir_1, '')
# os.makedirs(model_dir, exist_ok=True)

In [22]:
def main():
    log.write('>> parameter:\n{}\nargs:{}\n'.format(parameter, args), is_terminal=0)
    # set random seeds
    seed_everything(parameter.random_seed)
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    log.write('>> data_processing...\n')
    
    # debug = not os.getenv('KAGGLE_IS_COMPETITION_RERUN')
    #     if debug:
    #         return 1
    # load data
    
    # train_df, code_df_valid = preprocess.get_data()
    train_df = pd.read_parquet(parameter.data_dir+'train_df')
    code_df_valid = pd.read_parquet(parameter.data_dir+'code_df_valid')
    oof_prediction = np.zeros((len(train_df)))
    
    eval_loss = []
    log.write('---------------------------------------------------------------------------------------------------\n')
    for fold in range(parameter.k_folds):
        if len(args.folds) > 0 and str(fold) not in args.folds.split(','):
            continue
        model = MarkdownModel(utils.get_model_path(args.model_name, res = parameter.model_dir), seq_length=parameter.seq_length, pretrained=True)
        model.zero_grad()
        if parameter.use_cuda:
            model = model.cuda()
            # p =>
            # Parameter containing:
            # tensor([[-0.0504, -0.0459, -0.0496,  ..., -0.0528, -0.0523, -0.0369],
            #         [-0.0017, -0.0081,  0.0130,  ..., -0.0137, -0.0064, -0.0035],
            #         ...,
            #         [-0.0448, -0.0498, -0.0464,  ..., -0.0550, -0.0567, -0.0458],
            #         [-0.0400, -0.0428, -0.0420,  ..., -0.0588, -0.0488, -0.0336]],
            #        requires_grad=True)
            # p.numel() => 131174400
        total_params = sum(p.numel() for p in model.parameters())        
            # total_params => 450807809

        log.write('model total parameters:{}\n'.format(total_params), is_terminal=0)
        total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            # total_trainable_params => 450807809
        log.write('model total training parameters:{}\n'.format(total_trainable_params), is_terminal=0)
        tokenizer = AutoTokenizer.from_pretrained(utils.get_model_path(args.model_name, res = parameter.model_dir))

        train_model_filename = os.path.join(model_dir_seg, args.model_name + '_fold{}_fnl.pth.tar'.format(fold))
            # train_model_filename => models/deberta-v3-large_fold0.pth.tar
        if not os.path.exists(train_model_filename):
            # model.load_state_dict(torch.load(train_model_filename, map_location='cpu')['state_dict'])
            filename = os.path.join(model_dir_seg, args.model_name + '_fold{}.pth.tar'.format(fold))
            if os.path.exists(filename):
#                 model.encoder.load_state_dict(torch.load(filename)['state_dict'])
                model.load_state_dict(torch.load(filename)['state_dict'], strict=False)                
                log.write('loaded pre-trained model\n')
            else:
                log.write('not loaded pre-trained model\n')

            collate_fn = Collate(tokenizer)
            train_dataset = MarkdownDataset(train_df, tokenizer, fold, mode='train', parameter=parameter)
            train_loader = DataLoader(train_dataset, shuffle=True, batch_size=args.batch_size,
                                      num_workers=args.workers, pin_memory=True, worker_init_fn=utils.worker_init_fn,
                                      collate_fn=collate_fn)
            valid_dataset = MarkdownDataset(train_df, tokenizer, fold, mode='valid', parameter=parameter)
            valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=args.batch_size,
                                      num_workers=args.workers, pin_memory=True, collate_fn=collate_fn)
            criterion = utils.MyLoss()  # MyBCELoss()  # MyLoss()

            best_score = -1
            patience_cnt = 0
            is_improved = True
            optimizer = AdamW(model.parameters(), lr=args.lr, betas=(0.9, 0.999), eps=1e-6, weight_decay=args.weight_decay)
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=args.base_epoch, eta_min=args.eta_min)#args.lr / 5
            # scheduler = get_constant_schedule_with_warmup(optimizer, 100)
            # scheduler = get_cosine_schedule_with_warmup(optimizer, 100, num_training_steps= len(train_loader))
            # log.write('not loaded trained model\n')
            scaler = torch.cuda.amp.GradScaler(enabled=True)
            for epoch in range(args.base_epoch):
                # adjust learning rate for each epoch
                # adjust_learning_rate(optimizer, epoch, parameter)
                scheduler.step(epoch=epoch)
                log.write('Fold: [{0}] Epoch: [{1}], lr:[{2}]\n'.format(fold, epoch, optimizer.param_groups[0]['lr']))
                is_adversial = False
                train_loss, train_batch_time = train(model, train_loader, criterion, scheduler, optimizer, epoch,
                                                     args.n_accumulate, is_adversial, scaler)
                log.write('Epoch avg loss: {0}, Epoch cost time:{1} min\n'.format(train_loss, train_batch_time / 60))
                with torch.no_grad():                    
                    utils.save_model(model, model_dir, '{}_fold{}'.format(args.model_name, fold))                    
                    valid_loss, valid_batch_time = validate(model, criterion, valid_loader)#, code_df_valid)
                    log.write('Epoch avg loss: {0}, Epoch cost time:{1} min\n'.format(valid_loss, valid_batch_time / 60))                    
#                     rank_pred, score = validate(model, valid_loader, code_df_valid)
#                     log.write('Epoch score: {0}\n'.format(score))
#                     if score > -1:  # best_score:
#                         best_score, best_epoch = score, epoch
#                         # oof_prediction[np.where(train_df['fold_flag'] == fold)] = rank_pred.copy()
#                         utils.save_model(model, model_dir, '{}_fold{}'.format(args.model_name, fold))
#                         log.write('********Best Epoch: [{0}], Best Score:{1}********\n'.format(best_epoch, best_score))
#                     else:
#                         is_improved = False
#                         patience_cnt += 1
        else:
            valid_dataset = MarkdownDataset(train_df, tokenizer, fold, mode='valid', parameter=parameter)
            valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=args.batch_size * 4,
                                      num_workers=args.workers, pin_memory=True)
            model.load_state_dict(torch.load(train_model_filename, map_location='cpu')['state_dict'], strict=False)
            model.cuda()
            log.write('loaded trained model\n')
            with torch.no_grad():
                rank_pred, score = validate(model, valid_loader, code_df_valid)
            log.write('Epoch score: {0}\n'.format(score))
            best_score = score
            # oof_prediction[np.where(train_df['fold_flag'] == fold)] = rank_pred.copy()
        # utils.save_model(model, model_dir,'{}_fold{}'.format(args.model_name, fold))
#         eval_loss.append(best_score)
        del model
        _ = gc.collect()
        torch.cuda.empty_cache()
#     log.write('CV mean:{} std:{}.'.format(np.mean(eval_loss), np.std(eval_loss)))
#     log.write('detail:{}'.format(np.round(eval_loss, 4)))
#     np.save(os.path.join(parameter.result_dir, args.model_name + '_oof.npy'), oof_prediction)


In [1]:
# model, valid_loader, code_df_valid = main()
main()